In [4]:
%load_ext autoreload
%autoreload 2

import warnings
import pickle
warnings.filterwarnings("ignore")

import sys
from os.path import join

import pandas as pd
import matplotlib.pylab as plt
import seaborn as sb
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from tqdm import tqdm
# custom imports
SCRIPTS_PATH = '../scripts'
if SCRIPTS_PATH not in sys.path:
    sys.path.append(SCRIPTS_PATH)
    
import data_preparation
from text_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd
dataset1_df = pd.read_csv('../data/new_data/dataset1.csv', index_col=0) # отзывы
dataset2_df = pd.read_csv('../data/new_data/dataset2.csv', index_col=0) # характеристики товаров

compare_v3_df = pd.read_csv('../data/new_data/compare_v3.csv', sep=';', index_col=0) # сравнения товаров 
views_df = pd.read_csv('../data/new_data/views.csv', sep=';', index_col=0) # просмотры (сессия) пользователей 

In [6]:
raw_text = list()

raw_text.extend(dataset1_df['BENEFITS'].dropna())
raw_text.extend(dataset1_df['DRAWBACKS'].dropna())

In [7]:
raw_sentences = list()

for text in tqdm(raw_text):
    raw_sentences.extend(map(lambda x: tokenize(x, drop_numbers=True), get_sentences(text)))
    
raw_sentences = [x for x in raw_sentences if len(x)>0]
len(raw_sentences)

100%|██████████| 103500/103500 [00:27<00:00, 3703.14it/s]


200551

## Gensim phrases

In [8]:
from gensim.summarization.textcleaner import clean_text_by_sentences, clean_text_by_word
from gensim.models.phrases import Phraser, Phrases

bigrams = Phrases(raw_sentences, threshold=10.)
bigram_phraser = Phraser(bigrams)
trigram = Phrases(bigram_phraser[raw_sentences], threshold=10.)
trigram_phraser = Phraser(trigram)

In [9]:
keywords = list()
for words in bigram_phraser[raw_sentences]:
    for word in words:
        if '_' in word:
            keywords.append(word)
            
keywords = [x.split('_') for x in set(keywords)]
len(keywords)

3349

In [10]:
for kw in keywords:
    if 'меню' in kw:
        print(kw)

['меню', 'настроек']
['навигация', 'меню']
['простое', 'меню']
['меню', 'простое']
['неудобное', 'меню']
['удобное', 'меню']
['понятное', 'меню']
['меню', 'русском']
['доступное', 'меню']


## FP-growth

In [11]:
import fim
patterns = fim.fpgrowth(raw_sentences, report = 'S', supp=0.008) # 0.008 - only BENEFITS
print(len([x for x in patterns if len(x[0])>1]))

for item in all_phrases:
    if 'обучения' in item:
        print(item)

2989


In [12]:
sorted([x for x in patterns if len(x[0])>1], key=lambda x: x[1], reverse=True)[:10]

[(('внешний', 'вид'), 0.7055561926891415),
 (('``', "''"), 0.5270479828073658),
 (('цена', 'качество'), 0.3295919741113233),
 (('сборки', 'качество'), 0.31014554901247066),
 (('красивый', 'дизайн'), 0.3066551650203689),
 (('хороший', 'звук'), 0.29718126561323555),
 (('удобно', 'очень'), 0.23086396976330212),
 (('хорошее', 'качество'), 0.21241479723362136),
 (('держит', 'долго'), 0.2064312818185898),
 (('большой', 'экран'), 0.20144501897273012)]

In [13]:
from datasketch import MinHash, MinHashLSH
lsh = MinHashLSH(threshold=0.5, num_perm=256) # threshold=0.5, num_perm=256

def get_hash(keywords):
    keywords = set(keywords)
    mhash = MinHash(num_perm=256)
    for item in keywords:
        mhash.update(item.encode('utf8'))
    return mhash

all_phrases = list()
# all_phrases.extend(raw_sentences)
all_phrases.extend([x[0] for x in patterns if len(x[0])>1])
all_phrases.extend(keywords)

for i, item in tqdm(enumerate(all_phrases), total=len(all_phrases)):
    lsh.insert(str(i), get_hash(item))

100%|██████████| 6338/6338 [00:10<00:00, 617.73it/s]


In [58]:
for idx in lsh.query(get_hash(set(tokenize('игра')))):
    print(all_phrases[int(idx)])

['вся', 'игра']
('вся', 'игра')
('крутая', 'игра')
('затягивает', 'игра')
('игра', 'хорошая')
('игра', 'графика')
('прикольная', 'игра')


In [91]:
TEST_PRODUCT_ID = 20022045

test_product = dataset1_df[dataset1_df['PRODUCT']==TEST_PRODUCT_ID]
test_product_all_text = '.'.join(list(test_product['TEXT']))
test_product_category_name = test_product['CATEGORY_NAME'].iloc[0]

extracted_keywords = list()
for sent in get_sentences(test_product_all_text):
    tokenized_sent = set(tokenize(sent))
    matches = lsh.query(get_hash(tokenized_sent))
    if len(matches)>0 and len(tokenized_sent)>1:
        extracted_keywords.append(sent.strip())
        print(sent.strip())

очень прост в обслуживании
все понятно интуитивно
вся пыль осталась в воде
все на своих местах
тест-драйв длился 30 минут
но пока покупкой доволен
а весь мусор накапливался просто в корпусе пылесоса
Ни во время уборки ни после
либо эйфория после покупки
фильтр пылесоса
держась одной рукой ручку
Купил несколько дней назад
Отличный пылесос
Пылесос отличный
длинный шнур
легко разбирается-собирается
легко моется
на мой взгляд хорошая
очень удобное управление на ручке
Что бы долго работал
Всем рекомендую
Купил полгода назад
Сейчас практически не тянет
Отличный пылесос
шерсть собирает влёт)))
регулировка мощности
Берите не пожалеете!
пылесос просто замечательный! Он очень юркий
Управление на ручке очень удобное
С уборкой теперь нет никаких проблем!
Покупка порадовала очень
Надеюсь радовать будет долго
Пылесос супер!!! Убирает отлично
очень легко моется
прост в обращении
Единственный недостаток немного тяжеловат
пластмасс дешевый
Пылесос отличный
Мощности хватает
Я не сожалею: пылесос хороший

In [ ]:
#category_keywords = pickle.load(open('../dumps/category_keywords.pkl', 'rb'))
#lsh = pickle.load(open('../dumps/minhash_lsh.pkl', 'rb'))
#stop_words = pickle.load(open('../dumps/stop_words.pkl', 'rb'))

In [44]:
import string, pickle
from datasketch import MinHash
import nltk

#category_keywords = pickle.load(open('../dumps/category_keywords.pkl', 'rb'))
#lsh = pickle.load(open('../dumps/minhash_lsh.pkl', 'rb'))
#stop_words = pickle.load(open('../dumps/stop_words.pkl', 'rb'))

class PhrasesExtractor:
    def __init__(self, minhash, category_keywords, stop_words):
        self.category_keywords = category_keywords
        self.minhash = minhash
        self.stop_words = stop_words
        self.punctuation = string.punctuation + '»«-–—`\'()'
    
    def tokenize(self, file_text, use_lower=True, drop_stopwords=True, use_stemmer=False, drop_numbers=False):
        tokens = nltk.word_tokenize(file_text)
        tokens = [x for x in tokens if ( x not in self.punctuation )]

        if use_lower:
            tokens = [w.lower() for w in tokens]

        if drop_numbers:
            tokens = [x for x in tokens if ( x not in string.digits )]

        if drop_stopwords:
            tokens = [x for x in tokens if ( x not in self.stop_words )]

        if use_stemmer:
            pass

        return tokens
    
    def get_sentences(self, text):
        if '.' in text:
            for sent in text.split('.'):
                if ',' in sent:
                    for sent_2 in sent.split(','):
                        if ' и ':
                            for sent_3 in sent_2.split(' и '):
                                yield sent_3.strip()
                        else:
                            yield sent_2.strip()
                else:
                    yield sent.strip()
        else:
            yield text
    
    def get_hash(self, keywords):
        keywords = set(keywords)
        mhash = MinHash(num_perm=256)
        for item in keywords:
            mhash.update(item.encode('utf8'))
        return mhash
    
    def get_category_keyword_score(self, category, keyword):
        category = category.strip()
        assert(category in self.category_keywords)
        current_category_keywords = self.category_keywords[category]
        if keyword in current_category_keywords:
            index = list(current_category_keywords.keys()).index(keyword)
            score = current_category_keywords[keyword]
            return index, score
        else:
            return -1, 0
    
    def filter_phrases_old(self, extracted_keywords, test_product_category_name):
        assighned_phrases = dict()
        for sent in extracted_keywords:
            for word in self.tokenize(sent):
                idx, current_score = self.get_category_keyword_score(test_product_category_name, word)
                if idx == -1:
                    continue
                if idx in assighned_phrases:
                    if current_score>assighned_phrases[idx][1]:
                        assighned_phrases[idx] = (sent, current_score)
                else:
                    assighned_phrases[idx] = (sent, current_score)
        return assighned_phrases
    
    def filter_phrases(self, extracted_keywords, test_product_category_name):
        assighned_phrases = dict()
        for sent in extracted_keywords:
            if 'купил' in sent:
                continue
            keywords = [self.get_category_keyword_score(test_product_category_name, word) for word in self.tokenize(sent)]
            idx, current_score = sorted(keywords, key=lambda x: x[1], reverse=True)[0]
            if idx == -1:
                continue
            if idx in assighned_phrases:
                if current_score>assighned_phrases[idx][1]:
                    assighned_phrases[idx] = (sent, current_score)
            else:
                assighned_phrases[idx] = (sent, current_score)
        return assighned_phrases
            
        
    def predict(self, df, top_n=5):
        test_product_all_text = '.'.join(list(df['TEXT']))
        test_product_category_name = df['CATEGORY_NAME'].iloc[0]

        extracted_keywords = list()
        for sent in self.get_sentences(test_product_all_text):
            tokenized_sent = set(self.tokenize(sent))
            matches = self.minhash.query(self.get_hash(tokenized_sent))
            if len(matches)>0 and len(tokenized_sent)>1:
                extracted_keywords.append(sent.strip())
        if top_n != None and len(extracted_keywords)>top_n:
            filtered_phrases = self.filter_phrases(extracted_keywords, test_product_category_name)
            filtered_phrases = [filtered_phrases[key][0] for key in sorted(filtered_phrases.keys())]
            return filtered_phrases[:top_n]
        else:
            return extracted_keywords

phrases_extractor = PhrasesExtractor(minhash=lsh, category_keywords=category_keywords, stop_words=stop_words)

test_product = dataset1_df[dataset1_df['PRODUCT']==30013578]
phrases_extractor.predict(test_product, top_n=None)

['Экран у него не такой четкий',
 'Батарея держит отлично',
 'Очень шустрый аппарат',
 'Цена завышена даже очень!!',
 'Всё отлично',
 'вот только батарейка быстро садится',
 ') 2) Фронтальная камера 2 МПикс',
 ') 3) Большой яркий экран 4',
 '(Лично мне нравится насыщенные цвета',
 '(Ну это как обычно у Samsung',
 '(Нормального размера! А то мучайся: бегай',
 '(Удобно лежит в руке',
 'лично мне удобно пользоваться одной рукой',
 'Задняя крышка настолько хрупкая',
 ') 3) Объём встроенной памяти',
 'Выход: прошивка',
 ') Комментарий: Лично я покупкой довольна',
 'но девайс меня всё равно впечатлил',
 'Фотографии хорошие',
 'Из недостатков: батарея',
 'Сразу после покупки',
 'когда я просто звонила',
 'Купил жене год назад',
 'Самый удобный андройд',
 'что телефон просто лежит в режиме ожидания!\nПри том',
 '1) Цвета не естественные',
 'Особенно зеленый',
 'Даже на максимальной яркости',
 'Не буду описывать плюсы',
 'Экран просто великолепный',
 'цвета яркие',
 'сенсор тоже очень порадовал

# Cells below are useless and werent included in solution

## Word2Vec

In [ ]:
clusters_keywords = list()

for sent in extracted_keywords:
    for word in sent:
        idx, score = get_category_keyword_score(test_product_category_name, word)

In [84]:
sent_vectors = list()

for sent in get_sentences(test_product_all_text):
    sent_preprocessed = set(tokenize(sent))
    matches = lsh.query(get_hash(sent_preprocessed))
    if len(matches)>0:
        word_vectors = [w2v[word] for word in sent_preprocessed if word in w2v]
        if len(word_vectors) == 0:
            word_vectors = [np.zeros(w2v_size)]
        sent_vectors.append((sent, np.mean(word_vectors, axis=0)))

In [85]:
from sklearn.cluster import DBSCAN, KMeans
N_CLUSTERS = min(len(sent_vectors), 8)

kmeans = KMeans(n_clusters=N_CLUSTERS)
kmeans.fit([x[1] for x in sent_vectors])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

## Select best item from cluster

In [86]:
from itertools import groupby

test_product_category_name = test_product['CATEGORY_NAME'].iloc[0]
sorted_clusters = sorted(zip(kmeans.labels_, sent_vectors), key=lambda x:x[0])

test_product_aggregated_result = dict()
for cluster, sentences in groupby(sorted_clusters, key=lambda x: x[0]):
    cluster_phrases = list()
    for cluster, (sent, w2v_vector) in sentences:
        sent_total_score = 0
        for word in tokenize(sent):
            word_cat_score = get_category_keyword_score(test_product_category_name, word)[1]
            sent_total_score += word_cat_score
        cluster_phrases.append((sent, sent_total_score))
    test_product_aggregated_result[cluster] = cluster_phrases

In [87]:
for cluster, sentences in enumerate(sorted(test_product_aggregated_result.values(),
                                 key=lambda x: sum(z[1] for z in x), reverse=True)):
    print(cluster)
    for sent, score in sorted(sentences, key=lambda x: x[1], reverse=True):
        print(' ',sent, score)

0
  Ни во время уборки ни после 0.359015470675
  Если убираетесь дома не каждый день 0.257453089684
  С уборкой теперь нет никаких проблем! 0.148038591659
  тест-драйв длился 30 минут 0.100685811633
  Через день 0.0462654816153
  Проработал год 0.0376872428525
  Пока не нравитс 0.0176919469655
  Что бы долго работал -0.00495883627949
  Сегодня купил -0.0126162482107
  Купил несколько дней назад -0.0347118618772
  Надеюсь радовать будет долго -0.0396582528018
  немного подождать -0.0499852659973
  Купил полгода назад -0.0603150919654
1
  Пылесос супер!!! Убирает отлично 0.264338987061
  шерсть собирает влёт))) 0.239768269764
  а весь мусор накапливался просто в корпусе пылесоса 0.234064298466
  Я не сожалею: пылесос хороший 0.112066078143
  Купил пылесос сегодня 0.100862658052
  что лучше мешкового пылесоса нет ничего 0.0841263157607
  у меня дома кот 0.0832005024348
  Купила пылесос недавно 0.0692756632316
  вся пыль осталась в воде 0.0663154645262
  Отличный пылесос 0.0640469611648
  